In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,56,2024-02-15,-1,122.236364,100.809091,0.574400,11.030909,21.794545,0.223527,120.569091,...,12.487273,21.780000,0.181691,108.734545,0,2,3,2,17,2024
1,53,2024-02-15,-1,119.346154,99.019231,0.556788,11.946154,27.430769,0.192385,119.186538,...,13.012727,29.098182,0.209800,117.490909,0,2,3,1,10,2024
2,55,2024-02-15,3,113.703704,96.722222,0.566759,12.892593,23.066667,0.222593,117.051852,...,12.432075,26.149057,0.197849,110.524528,0,2,3,2,18,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', show=True):
    if model_name == 'FCNN':
        fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
        fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)
        
        model_file = fcnns.iloc[0]['File']
        
        model = torch.load('Models/'+model_file)
        scaler = joblib.load('Models/Scalers/FCNN.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'FCNN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-15,MIL,MEM,1.142857,6.00,1
1,2024-02-15,GSW,UTA,1.800000,2.05,0
2,2024-02-15,MIN,POR,1.235294,4.30,1


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-15,MIL,MEM,1.142857,6.00,1
1,2024-02-15,GSW,UTA,1.800000,2.05,0
2,2024-02-15,MIN,POR,1.235294,4.30,1


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-15,MIL,MEM,1.142857,6.00,1
1,2024-02-15,GSW,UTA,1.800000,2.05,0
2,2024-02-15,MIN,POR,1.235294,4.30,1


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-15,MIL,MEM,1.142857,6.00,1
1,2024-02-15,GSW,UTA,1.800000,2.05,0
2,2024-02-15,MIN,POR,1.235294,4.30,1


In [12]:
makeModelPredictions(games_df, tonight_odds, 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-15,MIL,MEM,1.142857,6.00,1.0
1,2024-02-15,GSW,UTA,1.800000,2.05,0.0
2,2024-02-15,MIN,POR,1.235294,4.30,1.0
